In [2]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import datetime
import csv
import os
from sklearn.metrics import r2_score, mean_squared_error

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [1]:
from modules.nypd_data import read_orig_file
from modules.nypd_data import filter_raw_data
from modules.nypd_data import save_dated_felonies
from modules.nypd_data import load_dated_felonies
from modules.nypd_data import save_clean_felonies
from modules.nypd_data import load_clean_felonies
from modules.nypd_data import add_offense_category
from modules.nypd_data import add_datetime_columns
from modules.nypd_data import pivot_felonies
from modules.nypd_data import save_pivoted_felonies
from modules.nypd_data import load_pivoted_felonies

In [3]:
from modules.prediction import load_all_data
from modules.prediction import precrime_train_test_split
from modules.prediction import load_splits
from modules.prediction import create_all_splits
from modules.prediction import sample_model
from modules.poly_ridge import poly_ridge_model
from modules.fancy_time_series import fancy_time_series_model
from modules.eval_model import eval_predictions


In [8]:
from modules.nypd_data import load_clean_felonies
from modules.nyc_shapefiles import read_nyc_shapefiles
from modules.weather import get_precinct_centroids
from modules.weather import get_mean_latlon
from modules.weather import write_weather_data
from modules.weather import read_weather_data
from modules.weather import read_api_key

In [20]:
# Generating file for 2017
# Get the ytd crime file from the open data website and 
# store it in the folder precrime_data/raw
save_dated_felonies('../precrime_data/raw/'
                    , 'NYPD_Complaint_Data_Current_YTD.csv'
                   , '../precrime_data/dated_felonies_2017.csv')
dated_felonies = load_dated_felonies('../precrime_data/'
                                     , 'dated_felonies_2017.csv')
save_clean_felonies('../precrime_data/', 'dated_felonies_2017.csv'
                   , '../precrime_data/clean_felonies_2017.csv')
dated_felonies = dated_felonies[dated_felonies['REPORT_DATE'] >= '2017-01-01']
dated_felonies.to_csv('../precrime_data/clean_felonies_2017.csv')
nypd_data_2017 = load_clean_felonies('../precrime_data/'
                                     , 'clean_felonies_2017.csv')
add_offense_category(nypd_data_2017)
add_datetime_columns(nypd_data_2017)
pivoted_2017 = pivot_felonies(nypd_data_2017)
save_pivoted_felonies(nypd_data_2017, data_path='../precrime_data/'
                      , pivot_file='pivoted_felonies_2017.csv')

    

Starting (2017-12-05 15:00:31)...
Saving filtered output (2017-12-05 15:00:34)...
Done (2017-12-05 15:00:36)
Starting (2017-12-05 15:00:58)...
Done (2017-12-05 15:01:19)


In [25]:
# Creating weather data for 2017

nypd_data_2017 = load_clean_felonies('../precrime_data/'
                                     , 'clean_felonies_2017.csv')
precinct_dict, tract_dict, merged_census_info = read_nyc_shapefiles()
centroids = get_precinct_centroids(precinct_dict)
nyc_mean_latlon = get_mean_latlon(centroids)
nypd_dates = nypd_data_2017['COMPLAINT_DATETIME'].dt.date.unique()
hours = range(2,26,4)
darksky_api_key = read_api_key()
write_weather_data(nypd_dates, hours, darksky_api_key, nyc_mean_latlon
                  , filepath='../precrime_data/weather_hist_2017.csv')


HTTPError: Bad response